# Loading data into memory

Loading API is central to a lot of nilmtk operations and provides a great deal of flexibility. Let's look at ways in which we can load data from a NILMTK DataStore into memory. To see the full range of possible queries, we'll use the [iAWE data set](http://iawe.github.io) (whose HDF5 file can be downloaded [here](https://copy.com/C2sIt1UfDx1mfPlC)).

The `load` function returns a *generator* of DataFrames loaded from the DataStore based on the conditions specified. If no conditions are specified, then all data from all the columns is loaded.  (If you have not come across Python generators, it might be worth reading [this quick guide to Python generators](http://stackoverflow.com/a/1756156/732596).)

**NOTE**: If you are on Windows, remember to escape the back-slashes, use forward-slashs, or use raw-strings when passing paths in Python, e.g. one of the following would work:

```python
iawe = DataSet('c:\\data\\iawe\\iawe.h5')
iawe = DataSet('c:/data/iawe/iawe.h5')
iawe = DataSet(r'c:\data\iawe\iawe.h5')
```

In [1]:
from nilmtk import DataSet

iawe = DataSet('/data/iawe/iawe.h5')
elec = iawe.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='iAWE', site_meter, appliances=[])
  ElecMeter(instance=3, building=1, dataset='iAWE', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=4, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=5, building=1, dataset='iAWE', appliances=[Appliance(type='air conditioner', instance=2)])
  ElecMeter(instance=6, building=1, dataset='iAWE', appliances=[Appliance(type='washing machine', instance=1)])
  ElecMeter(instance=7, building=1, dataset='iAWE', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=8, building=1, dataset='iAWE', appliances=[Appliance(type='clothes iron', instance=1)])
  ElecMeter(instance=9, building=1, dataset='iAWE', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=10, building=1, dataset='iAWE', appliances=[A

Let us see what measurements we have for the fridge:

In [2]:
fridge = elec['fridge']
fridge.available_columns()

[('power factor', None),
 ('frequency', None),
 ('power', 'active'),
 ('power', 'reactive'),
 ('voltage', None),
 ('power', 'apparent'),
 ('current', None)]

## Loading data

### Load all columns (default)

In [3]:
df = next(fridge.load())
df.head()

physical_quantity          frequency  power              voltage    power  \
type                                 active reactive             apparent   
2013-06-07 05:30:00+05:30  50.070000  0.111    2.483  235.070007    2.486   
2013-06-07 05:30:01+05:30  50.080002  0.200    2.547  235.020004    2.555   
2013-06-07 05:30:02+05:30  50.080002  0.152    2.480  234.979996    2.485   
2013-06-07 05:30:03+05:30  50.060001  0.159    2.444  235.000000    2.449   
2013-06-07 05:30:04+05:30  50.060001  0.215    2.510  234.949997    2.519   

physical_quantity         current  
type                               
2013-06-07 05:30:00+05:30   0.011  
2013-06-07 05:30:01+05:30   0.011  
2013-06-07 05:30:02+05:30   0.011  
2013-06-07 05:30:03+05:30   0.010  
2013-06-07 05:30:04+05:30   0.011

### Load a single column of power data

Use `fridge.power_series()` which returns a generator of 1-dimensional `pandas.Series` objects, each containing power data using the most 'sensible' AC type:

In [4]:
series = next(fridge.power_series())
series.head()

2013-06-07 05:30:00+05:30    0.111
2013-06-07 05:30:01+05:30    0.200
2013-06-07 05:30:02+05:30    0.152
2013-06-07 05:30:03+05:30    0.159
2013-06-07 05:30:04+05:30    0.215
Name: (power, active), dtype: float32

or, to get reactive power:

In [5]:
series = next(fridge.power_series(ac_type='reactive'))
series.head()

2013-06-07 05:30:00+05:30    2.483
2013-06-07 05:30:01+05:30    2.547
2013-06-07 05:30:02+05:30    2.480
2013-06-07 05:30:03+05:30    2.444
2013-06-07 05:30:04+05:30    2.510
Name: (power, reactive), dtype: float32

### Specify physical_quantity or AC type

In [6]:
df = next(fridge.load(physical_quantity='power', ac_type='reactive'))
df.head()

physical_quantity,power
type,reactive
2013-06-07 05:30:00+05:30,2.483
2013-06-07 05:30:01+05:30,2.547
2013-06-07 05:30:02+05:30,2.480
2013-06-07 05:30:03+05:30,2.444
2013-06-07 05:30:04+05:30,2.510


To load voltage data:

In [7]:
df = next(fridge.load(physical_quantity='voltage'))
df.head()

physical_quantity,voltage
type,
2013-06-07 05:30:00+05:30,235.070007
2013-06-07 05:30:01+05:30,235.020004
2013-06-07 05:30:02+05:30,234.979996
2013-06-07 05:30:03+05:30,235.000000
2013-06-07 05:30:04+05:30,234.949997


In [8]:
df = next(fridge.load(physical_quantity = 'power'))
df.head()

physical_quantity          power                  
type                      active reactive apparent
2013-06-07 05:30:00+05:30  0.111    2.483    2.486
2013-06-07 05:30:01+05:30  0.200    2.547    2.555
2013-06-07 05:30:02+05:30  0.152    2.480    2.485
2013-06-07 05:30:03+05:30  0.159    2.444    2.449
2013-06-07 05:30:04+05:30  0.215    2.510    2.519

### Loading by specifying AC type

In [9]:
df = next(fridge.load(ac_type='active'))
df.head()

physical_quantity,power
type,active
2013-06-07 05:30:00+05:30,0.111
2013-06-07 05:30:01+05:30,0.200
2013-06-07 05:30:02+05:30,0.152
2013-06-07 05:30:03+05:30,0.159
2013-06-07 05:30:04+05:30,0.215


### Loading by resampling to a specified period

In [10]:
# resample to minutely (i.e. with a sample period of 60 secs)
df = next(fridge.load(ac_type='active', sample_period=60))
df.head()

physical_quantity,power
type,active
2013-06-07 05:30:00+05:30,0.111000
2013-06-07 05:31:00+05:30,0.202000
2013-06-07 05:32:00+05:30,0.204000
2013-06-07 05:33:00+05:30,0.025000
2013-06-07 05:34:00+05:30,122.221001
